数据安全与隐私计算_作业_1_高洋洋_21312049 ：利用shamir门限方案实现图片的秘密共享

shamir秘密共享方案：将秘密分割成多个部分（称为份额）并将这些份额分发给参与者的加密方法

In [33]:
from random import randint
import random

secret = 254  # 要共享的秘密
num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量

# Shamir的秘密共享方案 - 秘密分割
def generate_coefficients(secret, threshold):
    coefficients = [secret] # 将秘密值作为系数列表的第一个元素
    for _ in range(threshold - 1):
        coefficients.append(random.randint(1, 100)) # 生成随机系数，添加到列表中
    return coefficients

def generate_shares(secret, num_shares, threshold):
    coefficients = generate_coefficients(secret, threshold)
    shares = []
    # coefficients = generate_coefficients(secret, threshold)
    for i in range(1, num_shares + 1):
        share = sum(c * (i ** power) for power, c in enumerate(coefficients)) % 256
        shares.append((i, share))
    return shares

# Shamir的秘密共享方案 - 秘密重构
def reconstruct_secret(shares):
    secret = 0
    for i, share in shares:
        num, den = 1, 1
        for j, _ in shares:
            if i != j:
                num *= -j
                den *= (i - j)
        lagrange = share * num / den
        secret += lagrange 
        secret = secret % 256
    return secret

# 将秘密分割成份额
shares = generate_shares(secret, num_shares, threshold)
print("份额:", shares)

# 使用最小份额阈值重构秘密
reconstructed_secret = reconstruct_secret(shares[:threshold])
print("重构的秘密:", reconstructed_secret)

份额: [(1, 178), (2, 130), (3, 12), (4, 238), (5, 198), (6, 50)]
重构的秘密: 254.0


需要传入图片，并将图片利用上述shamir方案进行秘密分割，之后再进行秘密恢复

当处理图像时，可以使用Python中的PIL或者OpenCV库来读取文件图像

In [34]:
from PIL import Image

def read_image(file_path):
    try:
        img = Image.open(file_path)
        return img
    except IOError:
        print("无法打开图像文件")

# 读取图像文件
file_path = "/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png"
image = read_image(file_path)

if image:
    print("图像读取成功")

图像读取成功


为了更好的测试代码，将输入的图片统一规格后再处理（不统一处理也可以，只是时间可能会长一点）

In [35]:
from PIL import Image
'''
# 打开第一张PNG图像
image1 = Image.open("/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png")
width, height = image1.size

# 打开第二张PNG图像
image2 = Image.open("/Users/colofuluangang/Desktop/code/Shamir/原始图片/2.png")

# 确保两张图像的尺寸相同
if image1.size != image2.size:
    image2 = image2.resize(image1.size)

# 确保两张图像的颜色模式相同（RGB）
if image1.mode != 'RGB':
    image1 = image1.convert('RGB')
if image2.mode != 'RGB':
    image2 = image2.convert('RGB')

# 保存image2
image2.save("/Users/colofuluangang/Desktop/code/Shamir/原始图片/resize2.png")
'''

'\n# 打开第一张PNG图像\nimage1 = Image.open("/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png")\nwidth, height = image1.size\n\n# 打开第二张PNG图像\nimage2 = Image.open("/Users/colofuluangang/Desktop/code/Shamir/原始图片/2.png")\n\n# 确保两张图像的尺寸相同\nif image1.size != image2.size:\n    image2 = image2.resize(image1.size)\n\n# 确保两张图像的颜色模式相同（RGB）\nif image1.mode != \'RGB\':\n    image1 = image1.convert(\'RGB\')\nif image2.mode != \'RGB\':\n    image2 = image2.convert(\'RGB\')\n\n# 保存image2\nimage2.save("/Users/colofuluangang/Desktop/code/Shamir/原始图片/resize2.png")\n'

利用数字签名算法，对图像进行数字签名，数字签名通过非对称加密算法实现，利用Python中的哈希算法和RSA数字签名算法来对图像进行数字签名，用来验证最后恢复的图像不是伪造的，也没有被修改，同样秘密分发者也无法抵赖图片是他发送的。

生成RSA公私钥对并将它们以文件形式保存，使用Python的Crypto库。其中公钥是公开的，私钥是秘密分发者个人持有，其他人无法得到

In [36]:
from Crypto.PublicKey import RSA

# 生成RSA密钥对
key = RSA.generate(2048)  # 这里使用2048位的密钥大小，您可以根据需要选择不同的密钥大小

# 获取私钥
private_key = key.export_key()

# 获取公钥
public_key = key.publickey().export_key()

# 保存私钥到文件
with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/private_key.pem", "wb") as private_key_file:
    private_key_file.write(private_key)

# 保存公钥到文件
with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/public_key.pem", "wb") as public_key_file:
    public_key_file.write(public_key)

对图像进行数字签名

In [37]:
from PIL import Image
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15

# 提取图像数据的像素数据
image = Image.open("/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png")
pixel_data = list(image.getdata())

# 计算图像数据的SHA-256哈希值
sha256_hash = SHA256.new()
for pixel in pixel_data:
    sha256_hash.update(bytes(pixel))

# 从密钥文件加载私钥
with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/private_key.pem", "rb") as private_key_file:
    private_key_data = private_key_file.read()

# 生成私钥对象
private_key = RSA.import_key(private_key_data)

# 使用私钥对SHA-256哈希值进行签名
signature_bytes = pkcs1_15.new(private_key).sign(sha256_hash)

# 保存数字签名
with open("/Users/colofuluangang/Desktop/code/Shamir/签名文件/signature.sig", "wb") as sig_file:
    sig_file.write(signature_bytes)
print("数字签名完成")


数字签名完成


利用Python的图像处理库来将RGB图像分成单独的R、G、B通道。

In [38]:
from PIL import Image 

def split_channels(image):
    r, g, b = image.split() #使用 split()函数拆分图像为R、G、B通道
    return r, g, b

# 读取图像文件
file_path = "/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png"
image = Image.open(file_path)
width, height = image.size

if image.mode == "RGB":
    print("图像为 RGB 模式")
    r_channel, g_channel, b_channel = split_channels(image)
    # 显示每个通道的图像
    r_channel.save("/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/R通道.png")
    g_channel.save("/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/G通道.png")
    b_channel.save("/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/B通道.png")
else:
    print("图像不是 RGB 模式")

图像为 RGB 模式


将图片分割成RGB三通道的灰度图片之后，获取每张图片的像素值，然后对每张图片进行shamir分割，然后得到分割后图片

In [39]:
from random import randint
import random
from PIL import Image
import numpy as np

num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量
file_path = "/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/B通道.png"
image = Image.open(file_path)
pixels = image.load()

def generate_coefficients(secret, threshold):
    coefficients = [secret] # 将秘密值作为系数列表的第一个元素
    for _ in range(threshold - 1):
        coefficients.append(random.randint(20, 300)) # 生成随机系数，添加到列表中
    return coefficients

# Shamir的秘密共享方案 - 秘密分割
images = []
def generate_shares(secret, num_shares, threshold, x, y):
    coefficients = generate_coefficients(secret, threshold)
    for i in range(1, num_shares + 1):
        share = sum(c * (i ** power) for power, c in enumerate(coefficients)) % 256
        images[i - 1].putpixel((x, y), share)


for i in range(num_shares):
    new_image = Image.new("L", (width, height))
    images.append(new_image)

for y in range(height):
    for x in range(width):
        generate_shares(pixels[x, y], num_shares, threshold, x, y)
        
for idx, img in enumerate(images):
    img.save(f"/Users/colofuluangang/Desktop/code/Shamir/B通道子秘密图片/B{idx}.png")
print(f"生成了{num_shares}个B通道子秘密灰度图像")


生成了6个B通道子秘密灰度图像


In [40]:
from random import randint
import random
from PIL import Image
import numpy as np

num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量
file_path = "/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/G通道.png"
image = Image.open(file_path)
pixels = image.load()

def generate_coefficients(secret, threshold):
    coefficients = [secret] # 将秘密值作为系数列表的第一个元素
    for _ in range(threshold - 1):
        coefficients.append(random.randint(20, 300)) # 生成随机系数，添加到列表中
    return coefficients


# Shamir的秘密共享方案 - 秘密分割
def generate_shares(secret, num_shares, threshold, x, y):
    coefficients = generate_coefficients(secret, threshold)
    for i in range(1, num_shares + 1):
        share = sum(c * (i ** power) for power, c in enumerate(coefficients)) % 256
        images[i - 1].putpixel((x, y), share)

images = []
for i in range(num_shares):
    new_image = Image.new("L", (width, height))
    images.append(new_image)

for y in range(height):
    for x in range(width):
        generate_shares(pixels[x, y], num_shares, threshold, x, y)
        
for idx, img in enumerate(images):
    img.save(f"/Users/colofuluangang/Desktop/code/Shamir/G通道子秘密图片/G{idx}.png")
print(f"生成了{num_shares}个G通道子秘密灰度图像")

生成了6个G通道子秘密灰度图像


In [41]:
from random import randint
import random
from PIL import Image
import numpy as np

num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量
file_path = "/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/R通道.png"
image = Image.open(file_path)
pixels = image.load()

def generate_coefficients(secret, threshold):
    coefficients = [secret] # 将秘密值作为系数列表的第一个元素
    for _ in range(threshold - 1):
        coefficients.append(random.randint(20, 300)) # 生成随机系数，添加到列表中
    return coefficients

# Shamir的秘密共享方案 - 秘密分割
def generate_shares(secret, num_shares, threshold, x, y):
    coefficients = generate_coefficients(secret, threshold)
    for i in range(1, num_shares + 1):
        share = sum(c * (i ** power) for power, c in enumerate(coefficients)) % 256
        images[i - 1].putpixel((x, y), share)

images = []
for i in range(num_shares):
    new_image = Image.new("L", (width, height))
    images.append(new_image)

for y in range(height):
    for x in range(width):
        generate_shares(pixels[x, y], num_shares, threshold, x, y)
        
for idx, img in enumerate(images):
    img.save(f"/Users/colofuluangang/Desktop/code/Shamir/R通道子秘密图片/R{idx}.png")
print(f"生成了{num_shares}个R通道子秘密灰度图像")

生成了6个R通道子秘密灰度图像


将分开的三通道灰度图合并为一张彩色图，发送给不同的参与方

In [42]:
from PIL import Image

num_shares = 6  # 要创建的份额总数

for i in range(num_shares):
    # 打开三张通道的图像
    r_channel = Image.open(f"/Users/colofuluangang/Desktop/code/Shamir/R通道子秘密图片/R{i}.png")
    g_channel = Image.open(f"/Users/colofuluangang/Desktop/code/Shamir/G通道子秘密图片/G{i}.png")
    b_channel = Image.open(f"/Users/colofuluangang/Desktop/code/Shamir/B通道子秘密图片/B{i}.png")

    # 合并图像
    merged_image = Image.merge("RGB", (r_channel, g_channel, b_channel))

    # 保存合并后的彩色图像
    merged_image.save(f"/Users/colofuluangang/Desktop/code/Shamir/子秘密图像/子秘密{i}.png")

利用不同通道的子秘密图像来恢复对应通道的秘密图像

In [43]:
from PIL import Image

new_image = Image.new("L", (width, height))
num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量

# 准备用于存储所有子图片数据的列表
images = []
for i in range(threshold):
    file_path = f"/Users/colofuluangang/Desktop/code/Shamir/B通道子秘密图片/B{i}.png"
    img = Image.open(file_path)
    images.append(img.load())  # 将像素数据加载到内存，避免多次读取文件

# Shamir的秘密共享方案 - 优化的秘密重构函数
def reconstruct_secret(images, x, y):
    secret = 0
    for i in range(1, threshold + 1):
        share = images[i - 1][x, y]
        num, den = 1, 1
        for j in range(1, threshold + 1):
            if i != j:
                num *= -j
                den *= (i - j)
        lagrange = share * num / den
        secret += lagrange
    secret %= 256
    return int(secret)

# 逐位置读取每张图片的灰度值并进行处理
for y in range(height):
    for x in range(width):
        new_pixels = reconstruct_secret(images, x, y)
        new_image.putpixel((x, y), new_pixels)
            
new_image.save("/Users/colofuluangang/Desktop/code/Shamir/B通道恢复秘密图片/B恢复.png")


In [44]:
from PIL import Image

num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量
new_image = Image.new("L", (width, height))

# 准备用于存储所有子图片数据的列表
images = []
for i in range(threshold):
    file_path = f"/Users/colofuluangang/Desktop/code/Shamir/G通道子秘密图片/G{i}.png"
    img = Image.open(file_path)
    images.append(img.load())  # 将像素数据加载到内存，避免多次读取文件

# Shamir的秘密共享方案 - 优化的秘密重构函数
def reconstruct_secret(images, x, y):
    secret = 0
    for i in range(1, 1 + threshold):
        share = images[i - 1][x, y]
        num, den = 1, 1
        for j in range(1, 1 + threshold):
            if i != j:
                num *= -j
                den *= (i - j)
        lagrange = share * num / den
        secret += lagrange
        secret %= 256
    return int(secret)

# 逐位置读取每张图片的灰度值并进行处理
for y in range(height):
    for x in range(width):
        new_pixels = reconstruct_secret(images, x, y)
        new_image.putpixel((x, y), new_pixels)
            
new_image.save("/Users/colofuluangang/Desktop/code/Shamir/G通道恢复秘密图片/G恢复.png")


In [45]:
from PIL import Image

num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量
new_image = Image.new("L", (width, height))

# 准备用于存储所有子图片数据的列表
images = []
for i in range(threshold):
    file_path = f"/Users/colofuluangang/Desktop/code/Shamir/R通道子秘密图片/R{i}.png"
    img = Image.open(file_path)
    images.append(img.load())  # 将像素数据加载到内存，避免多次读取文件

# Shamir的秘密共享方案 - 优化的秘密重构函数
def reconstruct_secret(images, x, y):
    secret = 0
    for i in range(1, 1 + threshold):
        share = images[i - 1][x, y]
        num, den = 1, 1
        for j in range(1, 1 + threshold):
            if i != j:
                num *= -j
                den *= (i - j)
        lagrange = share * num / den
        secret += lagrange
        secret %= 256
    return int(secret)

# 逐位置读取每张图片的灰度值并进行处理
for y in range(height):
    for x in range(width):
        new_pixels = reconstruct_secret(images, x, y)
        new_image.putpixel((x, y), new_pixels)
            
new_image.save("/Users/colofuluangang/Desktop/code/Shamir/R通道恢复秘密图片/R恢复.png")


最后将三通道的恢复灰度图像合并成彩色图像

In [46]:
from PIL import Image

# 打开三张通道的图像
r_channel = Image.open("/Users/colofuluangang/Desktop/code/Shamir/R通道恢复秘密图片/R恢复.png")
g_channel = Image.open("/Users/colofuluangang/Desktop/code/Shamir/G通道恢复秘密图片/G恢复.png")
b_channel = Image.open("/Users/colofuluangang/Desktop/code/Shamir/B通道恢复秘密图片/B恢复.png")

# 合并图像
merged_image = Image.merge("RGB", (r_channel, g_channel, b_channel))

# 保存合并后的彩色图像
merged_image.save("/Users/colofuluangang/Desktop/code/Shamir/恢复图像/恢复图像.png")


如果秘密发送者在场，他可以对比原始图片和恢复图片是否完全相同

In [47]:
from PIL import Image

# 打开第一张图片
image1 = Image.open("/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png")

# 打开第二张图片
image2 = Image.open("/Users/colofuluangang/Desktop/code/Shamir/恢复图像/恢复图像.png")

# 比较两张图片的像素数据
if list(image1.getdata()) == list(image2.getdata()):
    print("两张图片完全相同")
else:
    print("两张图片不同")


两张图片完全相同


如果没有秘密发送者在场，需要验证图片是原来的图片没有被篡改，则要验证签名，需要提取嵌入在图像中的数字签名数据，然后使用相应的公钥来验证签名的有效性

其中pkcs1_15.new(public_key).verify(sha256_hash, signature) 用来验证数字签名。使用 Crypto.Signature.pkcs1_15 模块中的 PKCS#1 v1.5 签名方案来验证数字签名的有效性。

1. public_key 是用于验证数字签名的公钥。
2. sha256_hash 是之前计算的图像数据的SHA-256哈希值。
3. signature 是数字签名本身，您希望验证其有效性。
4. 将公钥、哈希值和数字签名传递给 pkcs1_15.new(public_key).verify() 函数，该函数会验证数字签名是否与给定的哈希值相匹配。如果数字签名有效，那么函数将返回而不引发异常，否则将引发 ValueError 或 TypeError 异常，指示签名无效。

In [48]:
from PIL import Image
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15

# 打开数字签名文件
with open("/Users/colofuluangang/Desktop/code/Shamir/签名文件/signature.sig", "rb") as signature_file:
    signature = signature_file.read()

# 提取图像数据的像素数据
image = Image.open("/Users/colofuluangang/Desktop/code/Shamir/恢复图像/恢复图像.png")
pixel_data = list(image.getdata())

# 计算图像数据的SHA-256哈希值
sha256_hash = SHA256.new()
for pixel in pixel_data:
    sha256_hash.update(bytes(pixel))

# 打开公钥文件
with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/public_key.pem", "rb") as public_key_file:
    public_key_data = public_key_file.read()

# 创建RSA公钥对象
public_key = RSA.import_key(public_key_data)

try:
    # 使用公钥验证签名
    pkcs1_15.new(public_key).verify(sha256_hash, signature)
    print("数字签名验证成功，签名有效。")
except (ValueError, TypeError):
    print("数字签名验证失败，签名无效。")

数字签名验证成功，签名有效。
